<div align="center"><a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/Automatizacion/Automatización_2/clase_automatizacion_2_gsheets_cron_mails.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg'/> </a> <br> Recordá abrir en una nueva pestaña </div>

# **Automatización II: outputs, envío de mail, volcado a GSheets, scheduling**

### 🤖 Hoja de ruta 🤖

-   Ejemplo scrapeo sencillo con *Pandas*.					
-   Uso de la API de *Google Sheets*.							
-   Carga de datos en *Google Data Studio* y creación de un gráfico sencillo.
-   Ejemplo envío automático de mail con *smtplib*.				 	
-   Scheduling: cron para Mac y linux, GUI en Windows.					

### **Scrapping con Pandas**



Vamos a utilizar la librería de pandas para obtener las tablas que contiene una página. 

Documentación: https://pandas.pydata.org/docs/reference/api/pandas.read_html.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Datos de criptomonedas
url = "https://coinmarketcap.com/new/"

Ahora con *pandas.read_html()* es posible conseguir una lista con las tablas que contiene una *url*.

In [3]:
listadoTablas = pd.read_html(url)

In [4]:
listadoTablas[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                0 non-null      float64
 1   #                         30 non-null     int64  
 2   Name                      30 non-null     object 
 3   Price                     30 non-null     object 
 4   1h                        30 non-null     object 
 5   24h                       30 non-null     object 
 6   Fully Diluted Market Cap  30 non-null     object 
 7   Volume                    30 non-null     object 
 8   Blockchain                30 non-null     object 
 9   Added                     30 non-null     object 
 10  Unnamed: 10               0 non-null      float64
dtypes: float64(2), int64(1), object(8)
memory usage: 2.7+ KB


Seleccionamos la tabla de las criptomonedas recientemente añadidas.

In [5]:
df = listadoTablas[0].copy()
df.drop(['#','Unnamed: 10', 'Unnamed: 0'], axis=1, inplace=True) # Eliminamos columna valores nulos.
df.columns = ['Name', 'Price', 'PctChnge_1h', 'PctChnge_24h', 'FullDiluted_MarketCap', 'Volume', 'Blockchain', 'Added_HoursAgo']
df.replace('--', '0', inplace=True) # Imputamos valores nulos como np.nan
df.head(10)

,Name,Price,PctChnge_1h,PctChnge_24h,FullDiluted_MarketCap,Volume,Blockchain,Added_HoursAgo
0,Alpha Shiba Inu1ALPHASHIB,$0.000000001896,2.18%,81.38%,"$796,295","$464,536",Polygon,3 hours ago
1,Bit2me B2M Token2B2M,$0.1396,1.53%,6.67%,"$698,050,338","$6,757,193",Ethereum,4 hours ago
2,Lil Doge Floki3LDF,$0.00000005691,38.07%,10.93%,0,"$3,105,936",Binance Coin,6 hours ago
3,Skylight (New)4SLTN,$0.1654,1.29%,294.45%,"$82,723,983","$1,523,929",Binance Coin,7 hours ago
4,HobbsNetworkToken5HNW,$12.90,0.68%,0.86%,"$12,903,743","$506,404",Binance Coin,8 hours ago
5,Megacosm6MEGACOSM,$0.0000002419,0.34%,8.17%,"$241,906","$79,159",Ethereum,9 hours ago
6,P2P Solutions foundation7P2PS,$58.93,1.05%,1.82%,"$589,257,945,050","$943,409",Ethereum,9 hours ago
7,Baby Schrodinger Coin8BABYDINGER,$0.00000002323,4.83%,14.50%,0,"$565,222",Binance Coin,10 hours ago
8,SparkLab9Spark,$0.003181,1.41%,15.03%,0,"$5,222,271",Binance Coin,10 hours ago
9,SHKOOBY INU10SHKOOBY,$0.00000006953,4.64%,11.56%,"$69,531,226","$20,917,171",Ethereum,11 hours ago


In [6]:
# Convertimos a los tipos deseados
df['Price'] = df['Price'].replace( '[\$,)]','', regex=True ).astype(float)
df['FullDiluted_MarketCap'] = df['FullDiluted_MarketCap'].replace( '[\$,)]','', regex=True ).astype(float)
df['Volume'] = df['Volume'].replace( '[\$,)]','', regex=True ).astype(float)
df['PctChnge_1h'] = df['PctChnge_1h'].replace( '[\%,)]','', regex=True ).astype(float)/100
df['PctChnge_24h'] = df['PctChnge_24h'].replace( '[\%,)]','', regex=True ).astype(float)/100
df['Added_HoursAgo'] = df['Added_HoursAgo'].apply(lambda x: x[0])

In [7]:
df.head()

,Name,Price,PctChnge_1h,PctChnge_24h,FullDiluted_MarketCap,Volume,Blockchain,Added_HoursAgo
0,Alpha Shiba Inu1ALPHASHIB,1.896000e-09,0.0218,0.8138,796295.0,464536.0,Polygon,3
1,Bit2me B2M Token2B2M,1.396000e-01,0.0153,0.0667,698050338.0,6757193.0,Ethereum,4
2,Lil Doge Floki3LDF,5.691000e-08,0.3807,0.1093,0.0,3105936.0,Binance Coin,6
3,Skylight (New)4SLTN,1.654000e-01,0.0129,2.9445,82723983.0,1523929.0,Binance Coin,7
4,HobbsNetworkToken5HNW,1.290000e+01,0.0068,0.0086,12903743.0,506404.0,Binance Coin,8


## **Vinculación con Google Drive**

Para poder escribir/leer un archivo que se encuentra en *Google Drive*, será necesario primero, contar con un archivo de *autenticación* a su vez que compartir el archivo pertinente con el servicio creado desde Python.

#### Generación de archivo de autenticación. (Conexión local)

Para ello entraremos en el siguiente [link](https://console.cloud.google.com/apis/) ingresando con la cuenta de google que querramos vincular. Una vez dentro de la plataforma de Google Cloud, crearemos un proyecto. Una vez dentro del proyecto entraremos en la sección **API y servicios** y daremos click en la opción **habilitar API y servicio**.

  <img src="https://unket.s3.sa-east-1.amazonaws.com/static/gcp1.png" alt="drawing" width="500"/>

Allí dentro seleccionaremos el tipo de API que estemos necesitando. En este caso la de Google Drive. Una vez habilitada la API dentro de nuestro proyecto, iremos a la sección de **credenciales**, y dentro de la misma daremos click a **crear credenciales**. Seleccionamos la opción de *Cuenta de servicio*. Una vez que la *Cuenta de servicio* haya sido generada, será posible acceder a sus configuraciones y generar una clave en formato json dentro de la misma.

<img src="https://unket.s3.sa-east-1.amazonaws.com/static/gcp2.png" alt="drawing" width="500" height="300"/> <img src="https://unket.s3.sa-east-1.amazonaws.com/static/clave2.png" alt="drawing" width="500"/>

Más información sobre como crear un proyecto y habilitar una API [aquí](https://developers.google.com/workspace/guides/create-project)

In [8]:

# Esta funcion incluye todo lo que hicios antes, para poder actualizar nuestro DataFrame
def coinmarketcap_scraper():
  """
  Scraper de la pagina https://coinmarketcap.com/new/
  Obtiene los datos, los limpia y los devuelve como un DataFrame de Pandas.
  """
  url = "https://coinmarketcap.com/new/"
  
  # Scrapeamos la tabla con Pandas
  listadoTablas = pd.read_html(url)
  # creamos DataFrame
  df = listadoTablas[0].copy()

  # Limpieza de datos

  # Eliminamos columna valores nulos
  df.drop(['#','Unnamed: 10', 'Unnamed: 0'], axis=1, inplace=True) 
  df.columns = ['Name', 'Price', 'PctChnge_1h', 'PctChnge_24h', 'FullDiluted_MarketCap', 'Volume', 'Blockchain', 'Added_HoursAgo']
  # Imputamos valores nulos como np.nan 
  df.replace('--', '0', inplace=True) 

  # Convertimos a los tipos deseados
  df['Price'] = df['Price'].replace( '[\$,)]','', regex=True ).astype(float)
  df['FullDiluted_MarketCap'] = df['FullDiluted_MarketCap'].replace( '[\$,)]','', regex=True ).astype(float)
  df['Volume'] = df['Volume'].replace( '[\$,)]','', regex=True ).astype(float)
  df['PctChnge_1h'] = df['PctChnge_1h'].replace( '[\%,)]','', regex=True ).astype(float)/100
  df['PctChnge_24h'] = df['PctChnge_24h'].replace( '[\%,)]','', regex=True ).astype(float)/100
  df['Added_HoursAgo'] = df['Added_HoursAgo'].apply(lambda x: x[0])

  return df

### Usando gspread para interactuar con las hojas de cálculo de Google

Documentación: https://docs.gspread.org/en/v4.0.1/


In [9]:
# Instalamos y hacemos un upgrade de gspread porque la funcion que necesitamos esta a partir de la version 3.6
!pip install gspread --upgrade

  Attempting uninstall: gspread
    Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


Importamos la libreria gspread y chequeamos la version 

In [10]:
import gspread
print(f'Version de gspread:{gspread.__version__}')

Version de gspread:4.0.1


Autenticación local

In [16]:
# Local

gc = gspread.service_account(filename='credenciales.json') # Configurar con path de credenciales.

En caso de acceder desde Google Colab resulta más sencillo autenticarse dado que lo hace automaticamente con la cuenta linkeada.

In [11]:
# Colab
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

#### Interactuamos con Google Sheets.

Creamos una nueva hoja de calculo con su debido título y la compartimos con la cuenta desde la cual querramos acceder.

In [12]:
# Creamos la hoja de calculo
hoja_de_calculo = gc.create("CriptosRecientes")

# Para hacer visible el archivo es necesario compartirlo
tu_mail = 'tu_mail@gmail.com'
hoja_de_calculo.share(tu_mail, perm_type='user', role='writer')

In [13]:
# actualizamos el df
df = coinmarketcap_scraper()

In [14]:
# Obtenemos la pagina 0 de este archivo
primera_hoja = hoja_de_calculo.get_worksheet(0)

# Actualizo la hoja de calculo
primera_hoja.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '16P5SmNctJVUoXHQXdJyNjDv_3psMKKGECdJf1kYSHlQ',
 'updatedCells': 248,
 'updatedColumns': 8,
 'updatedRange': "'Hoja 1'!A1:H31",
 'updatedRows': 31}

Ahora probemos leer nuevamente el archivo para corroborar que se hayan cargado bien los datos.

In [15]:
# Obtenemos los valores de nuestra hoja de calculo
nuestra_hoja = primera_hoja.get_all_values()

# Cargamos con Pandas
df_aux = pd.DataFrame(nuestra_hoja)
df_aux.columns = df_aux.iloc[0,:]
df_aux = df_aux.iloc[1:,:]
df_aux.head()

,Name,Price,PctChnge_1h,PctChnge_24h,FullDiluted_MarketCap,Volume,Blockchain,Added_HoursAgo
1,Alpha Shiba Inu1ALPHASHIB,0.000000001896,0.0218,0.8138,796295,464536,Polygon,3
2,Bit2me B2M Token2B2M,0.1396,0.0153,0.0667,698050338,6757193,Ethereum,4
3,Lil Doge Floki3LDF,0.00000005691,0.3807,0.1093,0,3105936,Binance Coin,6
4,Skylight (New)4SLTN,0.1654,0.0129,2.9445,82723983,1523929,Binance Coin,7
5,HobbsNetworkToken5HNW,12.9,0.0068,0.0086,12903743,506404,Binance Coin,8


In [16]:
df_aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 1 to 30
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Name                   30 non-null     object
 1   Price                  30 non-null     object
 2   PctChnge_1h            30 non-null     object
 3   PctChnge_24h           30 non-null     object
 4   FullDiluted_MarketCap  30 non-null     object
 5   Volume                 30 non-null     object
 6   Blockchain             30 non-null     object
 7   Added_HoursAgo         30 non-null     object
dtypes: object(8)
memory usage: 2.0+ KB


In [ ]:
# Ejercicios: Scrapear los datos de las medallas obtenidas en gimnasia artistica femenina de esta entrada de wikipedia y guardarlos en una hoja de Google
# https://en.wikipedia.org/wiki/List_of_Olympic_medalists_in_gymnastics_(women)

## **Interacción con Google Data Studio**

<img src='https://www.mdmarketingdigital.com/blog/wp-content/uploads/2019/06/Data-Studio-Stats-1200x700.png' width=500>



[Google Data Studio](https://datastudio.google.com/) es una herramienta en línea para convertir datos en paneles e informes  personalizables

## **Envio automatico de e-mails**

En esta sección haremos un ejemplo de como enviar mails desde Python 

### **Protocolo SMTP**
El protocolo para transferencia simple de correo (en inglés: Simple Mail Transfer Protocol o SMTP) es un protocolo de red utilizado para el intercambio de mensajes de correo electrónico entre computadoras u otros dispositivos (PDA, teléfonos móviles, impresoras, etcétera).

### Usando Python

Para poder usar Python desde gmail tendremos que habilitar el uso de "aplicaciones poco seguras". 

<img src='https://docs.rocketbot.co/wp-content/uploads/2020/01/c3.png' >
<img src='https://docs.rocketbot.co/wp-content/uploads/2020/01/c4-768x244.png' >
<center>
Fuente: https://docs.rocketbot.co/?p=1567
</center>

* Si tienen problemas mirar aca: https://stackoverflow.com/questions/26852128/smtpauthenticationerror-when-sending-mail-using-gmail-and-python



In [62]:
import smtplib
from email.message import EmailMessage

msg = EmailMessage()

# Contenido
msg['From']="curso_de_automatizacion_de_humai@gmail.com"
msg['To']="tu_mail@gmail.com"
msg['Subject']= "Probando mandar mails!"
cuerpo_del_mail = 'Este es un mail enviado con Python en la clase! =D'
msg.set_content(cuerpo_del_mail)

# No se queden en los detalles aquí, pero pueden leer más sobre el protocolo SMTP acá: https://es.wikipedia.org/wiki/Protocolo_para_transferencia_simple_de_correo 
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()

# Usuario y contraseña
usuario = 'tu_usuario'
password = 'tu_contrasenia'

server.login(usuario, password)

# enviar
server.send_message(msg)
server.quit();

### Enviar más de un mail

Podría existir el caso de uso donde querramos enviar más de un mail. Por ejemplo a todes nuestres alumnes con la nota de su parcial.

In [3]:
import smtplib
from email.message import EmailMessage
import time

notas  = [10, 8 , 7]
alumnes = ['Juan', 'Sofia', 'Lupe']
mails = ['Juan@mail.com', 'Sofia@mail.com', 'Lupe@mail.com']

# Usuario y contraseña
usuario = 'tu_usuario'
password = 'tu_contrasenia'

with smtplib.SMTP('smtp.gmail.com', 587) as server:
  for i in range(len(notas)):
    # Contenido
    msg = EmailMessage()

    msg['From']="curso_de_automatizacion_de_humai@gmail.com"
    msg['To']="tu_mail@gmail.com" # Obviamente habria que ir variando los mails, aca no lo voya hacer pero seria poner mails[i]
    msg['Subject']= "Probando mandar mails!"
    cuerpo_del_mail = f'Hola {alumnes[i]}, tu nota en el parcial fue de {notas[i]}.\n\nSaludos!'
    msg.set_content(cuerpo_del_mail)
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(usuario, password)
    # server.starttls()

    # enviar
    server.send_message(msg)
    time.sleep(3)
    print(f'mail enviado a {alumnes[i]}')

mail enviado a Juan
mail enviado a Sofia
mail enviado a Lupe


### Enviar archivos adjuntos

Podemos agregar archivos adjuntos como por ejemplo imágenes o PDFs.

In [4]:
# Enviar archivos adjuntos

import smtplib
# El módulo imghdr determina el tipo de imagen contenida en un archivo.
import imghdr
from email.message import EmailMessage

msg = EmailMessage()

# Contenido
msg['From']="curso_de_automatizacion_de_humai@gmail.com"
msg['To']="tu_mail@gmail.com"
msg['Subject']= "Probando mandar mails!"
cuerpo_del_mail = 'Te estoy enviando una imagen con Python! =D'
msg.set_content(cuerpo_del_mail)

path_imagen = '/content/humai_logo.png' 

with open(path_imagen, 'rb') as f:
    image_data = f.read()
    # Para saber el tipo de archivo
    image_type = imghdr.what(f.name)
    image_name = f.name

msg.add_attachment(image_data, maintype='image', subtype=image_type, filename=image_name)

# No se queden en los detalles aquí, pero pueden leer más sobre el protocolo SMTP acá: https://es.wikipedia.org/wiki/Protocolo_para_transferencia_simple_de_correo 
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()

# Usuario y contraseña
usuario = 'tu_usuario'
password = 'tu_contrasenia'

server.login(usuario, password)

# enviar
server.send_message(msg)
print('Mail enviado')
server.quit();


Mail enviado


In [ ]:
# Ejercicios: 
# Enviar tres mails que contengan un PDF

## Scheduling con CRON

En el sistema operativo Unix, cron es un administrador regular de procesos en segundo plano (demonio) que ejecuta procesos o guiones a intervalos regulares (por ejemplo, cada minuto, día, semana o mes). Los procesos que deben ejecutarse y la hora a la que deben hacerlo se especifican en el archivo crontab. 

<img src="https://i.ibb.co/ZWCbc2m/crontab.png" alt="crontab" border="0">


Cron se puede definir como el equivalente a Tareas Programadas de Windows.

<img src='https://www.solvetic.com/uploads/monthly_01_2017/tutorials-9832-0-90051600-1484655732.png'> <br>
Fuente: https://www.solvetic.com/tutoriales/article/3441-como-abrir-y-configurar-programador-tareas-windows-10/





### Comandos básicos de Cron

En la terminal: <br>
**`crontab -l`** -> Permite ver la lista de las tareas programadas <br>
**`crontab -e`** -> Permite ver editar las tareas programadas

<font color='red'><h3>ATENCION! </h3></font>

**`crontab -r`** -> Permite borrar las tareas programadas  <br>
Es importante que lo uses cuando quieras que tu tarea deje de ser ejecutada, sino va a quedar funcionando indefinidamente 

In [ ]:
# ┌───────────── Minutos (0 - 59)
# │ ┌───────────── Hora (0 - 23)
# │ │ ┌───────────── Dia del mes (1 - 31)
# │ │ │ ┌───────────── Mes (1 - 12) o jan,feb,mar,apr,may,jun,jul... (meses en inglés)
# │ │ │ │ ┌─────────────  día de la semana (0-6) (domingo=0 o 7) o sun,mon,tue,wed,thu,fri,sat (días en inglés) 
# │ │ │ │ │                                       
# │ │ │ │ │
# │ │ │ │ │
# * * * * *  comando_a_ejecutar

Algunos ejemplos:

Todos los dias a las 12 y media del mediodia corre esto

`30 12 * * * python /ruta/a/mi/archivo/script.py`

El 10 de cada mes corre esto a las 3 de la tarde

` * 3 10 * * python /ruta/a/mi/archivo/script.py`

Consideraciones
* Dependiendo del intérprete de Python que tengan instalado pueden tener que poner algo distinto a la palabra python. Ejemplo:
` * * * * * python3 /ruta/a/mi/archivo/script.py`

* Otra forma es que cron se posicione en la ruta del archivo y luego solo lo corra. Ejemplo:
`* * * * * cd /ruta/a/mi/archivo && python script.py`


In [ ]:
def escribir_archivo():
	with open('prueba.txt', 'a+') as f:
		f.write('Esribiendo archivo desde Cron\n')
		
if __name__ == '__main__':
	escribir_archivo()

In [ ]:
# Ejercicio para hacer con CRON o con el programador de tareas de windows:
# Enviarse un mail a ustedes mismos cada 8 horas diciendo que van a avanzar con Python y lograr sus objetivos :)

### Recursos

* Google Data Studio
  * https://datastudio.google.com/gallery les recomendamos ver la galeria para inspirarse y ver todo lo que se puede hacer con esta herramienta


* cron: 
  * [Video de Corey Schafer](https://www.youtube.com/watch?v=QZJ1drMQz1A&t=12s&ab_channel=CoreySchafer) , aunque esta en ingles se los super recomiendo
  * https://crontab.guru/ Ayuda a escribir los comandos de cron

